In [1]:
import os
import pandas as pd
from src.processer_file import read_path as reader
from src.processer_file import get_path 
from src.processer_file import export_file

# export_file.export_to_csv(15, 'task')
# df_dict = reader.read(16, 'task')

# for k,v in df_dict.items():
#     print(k)
#     for k2, v2 in v.items():
#         print(f'{k2, v2.shape}')

export_file.export_to_csv(16, 'task')

Created directory: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\CK\TASKData
File to small (0, 30), ck20240321 has no info to extract.
Too litte info to extract.
Created directory: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\ICA\TASKData
File exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\ICA\TASKData\ica20240405.csv
Weekly file exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\ICA\TASKData\ICA_W16_Task.csv

Created directory: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\TJ\TASKData
File exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\TJ\TASKData\TJ0328_0329KPI.csv
Weekly file exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\TJ\TASKData\TJ_W16_Task.csv

No of tasks: 152



c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\src\processer_data\task_process.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet1['Time Gap'] = df_form_dict[['navigation_dur(s)', 'alignment_dur(s)','opera_dur(s)', 'gap_navi_to_align(s)', 'gap_align_to_oper(s)']].apply(list, axis=1)
c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\src\processer_data\task_process.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet1['checkpoint'] = [['navigation_duration', 'alignme

Task excel file exported to c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\TJ\TASKData\TJ_W16_Task.xlsx.
Created directory: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\WH\TASKData
File exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\WH\TASKData\好运达.csv
File exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\WH\TASKData\海通18.csv
Weekly file exported to: c:\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W16\WH\TASKData\WH_W16_Task.csv

